 <p style="text-align: center;">ETL REVIEWS.</p>

##### Cargo librerias.

In [23]:
# Realizo carga de librerias.

import ast
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')
from nltk.sentiment import SentimentIntensityAnalyzer
import re

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Arnaldo\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Arnaldo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Cargo el archivo a normalizar y lo transformo en un Dataframe.

In [24]:
filas = list()
with open(r"D:\Users\Arnaldo\Desktop\SISTEMAS\SOYHENRY\CURSO\PROYECTOS\PROYECTO INDIVIDUAL I\RECURSADO\ARCHIVOS\ARCHIVOS JSON\australian_user_reviews.json", "r", encoding="Latin-1") as archivo:
    for linea in archivo.readlines():
        filas.append(ast.literal_eval(linea))

df_reviews = pd.DataFrame(filas)
df_reviews



,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


Hago un informe del dataframe.

In [25]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


Verifico datos anidados y los normalizo.

In [26]:
def verificar_columna(column):
    tiene_anidados = any(isinstance(valor, (list, dict, tuple,set)) for valor in column)
    tipos_datos = set(type(valor).__name__ for valor in column)
    return tiene_anidados, tipos_datos

# Verificar cada columna especificada
for col in df_reviews:
    tiene_anidados, tipos_datos = verificar_columna(df_reviews[col])
    if tiene_anidados:
        print(f'La columna "{col}" tiene datos anidados')
        print(f'Tipos de datos en la columna "{col}": {tipos_datos}')
    else:
        print(f'La columna "{col}" no tiene datos anidados')
        print(f'Tipos de datos en la columna "{col}": {tipos_datos}')
      
    

La columna "user_id" no tiene datos anidados
Tipos de datos en la columna "user_id": {'str'}
La columna "user_url" no tiene datos anidados
Tipos de datos en la columna "user_url": {'str'}
La columna "reviews" tiene datos anidados
Tipos de datos en la columna "reviews": {'list'}


In [27]:
reviewsdf_expandido = df_reviews.explode('reviews')
reviewsdf_desanidado = pd.json_normalize(reviewsdf_expandido['reviews'])

reviewsdf_expandido = reviewsdf_expandido.reset_index(drop=True)
reviewsdf_desanidado = reviewsdf_desanidado.reset_index(drop=True)

df_reviews_final = pd.concat([reviewsdf_expandido, reviewsdf_desanidado], axis=1)
df_reviews_final.head(5)

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [28]:
df_reviews_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59333 non-null  object
 1   user_url     59333 non-null  object
 2   reviews      59305 non-null  object
 3   funny        59305 non-null  object
 4   posted       59305 non-null  object
 5   last_edited  59305 non-null  object
 6   item_id      59305 non-null  object
 7   helpful      59305 non-null  object
 8   recommend    59305 non-null  object
 9   review       59305 non-null  object
dtypes: object(10)
memory usage: 4.5+ MB


#

In [29]:
df_reviews_final.drop(['user_url', 'funny', 'posted', 'last_edited', 'reviews', 'helpful'], axis=1, inplace=True)
df_reviews_final

,user_id,item_id,recommend,review
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,22200,True,It's unique and worth a playthrough.
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,251610,True,I know what you think when you see this title ...
4,js41637,227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...
59328,76561198312638244,70,True,a must have classic from steam definitely wort...
59329,76561198312638244,362890,True,this game is a perfect remake of the original ...
59330,LydiaMorley,273110,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,730,True,:D


##### Elimino filas con nulos.

In [30]:
df_reviews_final = df_reviews_final.dropna(how='all')
df_reviews_final

,user_id,item_id,recommend,review
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,22200,True,It's unique and worth a playthrough.
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,251610,True,I know what you think when you see this title ...
4,js41637,227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...
59328,76561198312638244,70,True,a must have classic from steam definitely wort...
59329,76561198312638244,362890,True,this game is a perfect remake of the original ...
59330,LydiaMorley,273110,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,730,True,:D


#### Elimino posibles duplicados.

In [31]:
df_reviews_final.drop_duplicates(subset=None, keep='first', inplace=False)
df_reviews_final

,user_id,item_id,recommend,review
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,22200,True,It's unique and worth a playthrough.
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,251610,True,I know what you think when you see this title ...
4,js41637,227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...
59328,76561198312638244,70,True,a must have classic from steam definitely wort...
59329,76561198312638244,362890,True,this game is a perfect remake of the original ...
59330,LydiaMorley,273110,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,730,True,:D


#### Reseteo indices.

In [32]:
df_reviews_final.reset_index(drop=True, inplace=True)

#### Ajusto el campo review.

In [34]:
#Se convierten todo el texto en minúsculas.
df_reviews_final['review']= df_reviews_final['review'].str.lower()
#Se elimina cualquier carácter que no sea una letra, número o espacio en blanco con una cadena vacía
df_reviews_final['review']= df_reviews_final['review'].str.replace('[^\w\s]', '', regex=True)
#Se convierten todos los valores a str para su tokenización
df_reviews_final['review'] = df_reviews_final['review'].astype(str)

In [35]:
df_reviews_final

,user_id,item_id,recommend,review
0,76561197970982479,1250,True,simple yet with great replayability in my opin...
1,76561197970982479,22200,True,its unique and worth a playthrough
2,76561197970982479,43110,True,great atmosphere the gunplay can be a bit chun...
3,js41637,251610,True,i know what you think when you see this title ...
4,js41637,227300,True,for a simple its actually not all that simple ...
...,...,...,...,...
59328,76561198312638244,70,True,a must have classic from steam definitely wort...
59329,76561198312638244,362890,True,this game is a perfect remake of the original ...
59330,LydiaMorley,273110,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,730,True,d


#### Genero un nuevo dataframe para aplicarlo a las funciones.

In [36]:
df_reviews_coments = df_reviews_final.copy()

#### Realizo el anallisis de sentimientos.

In [37]:
#Se define una función para realizar el analisis de sentimiento.
def analizar_sentimiento(texto):
    if isinstance(texto, str):
        if texto.strip() != '':

            sia = SentimentIntensityAnalyzer()
            sentimiento = sia.polarity_scores(texto)["compound"]

            if sentimiento > 0.2:
                return 2
            elif sentimiento < -0.2:
                return 0

            else: return 1

    return 1
df_reviews_final["sentiment_analysis"] = df_reviews_final["review"].fillna('').apply(lambda i: analizar_sentimiento(i))

In [38]:
df_reviews_final

,user_id,item_id,recommend,review,sentiment_analysis
0,76561197970982479,1250,True,simple yet with great replayability in my opin...,2
1,76561197970982479,22200,True,its unique and worth a playthrough,2
2,76561197970982479,43110,True,great atmosphere the gunplay can be a bit chun...,2
3,js41637,251610,True,i know what you think when you see this title ...,2
4,js41637,227300,True,for a simple its actually not all that simple ...,2
...,...,...,...,...,...
59328,76561198312638244,70,True,a must have classic from steam definitely wort...,2
59329,76561198312638244,362890,True,this game is a perfect remake of the original ...,2
59330,LydiaMorley,273110,True,had so much fun plaing this and collecting res...,2
59331,LydiaMorley,730,True,d,1


#### Con el campo 'sentiment_analysis' anexado al dataframe procedo a eliminar la columna review

In [39]:
df_reviews_final.drop(['review'], axis=1, inplace=True)
df_reviews_final

,user_id,item_id,recommend,sentiment_analysis
0,76561197970982479,1250,True,2
1,76561197970982479,22200,True,2
2,76561197970982479,43110,True,2
3,js41637,251610,True,2
4,js41637,227300,True,2
...,...,...,...,...
59328,76561198312638244,70,True,2
59329,76561198312638244,362890,True,2
59330,LydiaMorley,273110,True,2
59331,LydiaMorley,730,True,1


In [40]:
df_reviews_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59333 non-null  object
 1   item_id             59305 non-null  object
 2   recommend           59305 non-null  object
 3   sentiment_analysis  59333 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.8+ MB


In [41]:
df_reviews_coments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59333 non-null  object
 1   item_id    59305 non-null  object
 2   recommend  59305 non-null  object
 3   review     59333 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


Procedo a guardar los archivos en formato CSV y Parquet para su posterior uso en la API.

In [42]:
df_reviews_coments.to_csv(r'D:\Users\Arnaldo\Desktop\SISTEMAS\SOYHENRY\CURSO\PROYECTOS\PROYECTO INDIVIDUAL I\RECURSADO\ARCHIVOS\DATAFRAMES\df_reviews_coments.csv', index=False)
df_reviews_final.to_csv(r'D:\Users\Arnaldo\Desktop\SISTEMAS\SOYHENRY\CURSO\PROYECTOS\PROYECTO INDIVIDUAL I\RECURSADO\ARCHIVOS\DATAFRAMES\df_reviews.csv', index=False)

In [43]:
df_reviews_final.to_parquet(r'D:\Users\Arnaldo\Desktop\SISTEMAS\SOYHENRY\CURSO\PROYECTOS\PROYECTO INDIVIDUAL I\RECURSADO\ARCHIVOS\DATAFRAMES\DATAFRAMES PARKET\df_reviews.parquet')
df_reviews_coments.to_parquet(r'D:\Users\Arnaldo\Desktop\SISTEMAS\SOYHENRY\CURSO\PROYECTOS\PROYECTO INDIVIDUAL I\RECURSADO\ARCHIVOS\DATAFRAMES\DATAFRAMES PARKET\df_reviews_coments.parquet')

c:\Users\Arnaldo\anaconda3\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
